In [2]:
import sys
sys.path.append('..')
import feature_spaces
import encoding_utils
import dvu
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
import seaborn as sns
import os
import pandas as pd
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import RidgeCV, LogisticRegressionCV

from typing import List
from matplotlib import pyplot as plt
from os.path import join
import numpy as np
import datasets
import encoding_utils, feature_spaces, imodelsx
from spacy.lang.en import English
import sklearn.preprocessing
import pickle as pkl
%load_ext autoreload
%autoreload 2

from feature_spaces import *
from feature_spaces import em_data_dir, data_dir, results_dir
# from fit_linear_models import get_dsets
# from ridge_utils.SemanticModel import SemanticModel
dvu.set_style()

### Load story texts

In [3]:
# Look at narrative stories
train_stories, test_stories, allstories = encoding_utils.get_allstories([1, 2, 3, 4, 5])
wordseqs = feature_spaces.get_story_wordseqs(train_stories)
texts = [' '.join(wordseqs[k].data) for k in wordseqs.keys()]

In [8]:
with open('narrative_stories.txt', 'w') as f:
    f.write('\n'.join(texts))

### Save prepocessor

In [ ]:
# fit an fMRI preprocessor
narrative_stories = texts
# print(len(narrative_stories)) # 26 stories
ngrams_lists = [
    imodelsx.util.generate_ngrams_list(story, ngrams=8,
                                    all_ngrams=False, tokenizer_ngrams=English().tokenizer)
    for story in narrative_stories
]

# merge all the lists in ngrams_lists
ngrams_list = sum(ngrams_lists, [])
embs = imodelsx.util.get_embs_llm(ngrams_list, 'bert-base-uncased')
preproc = sklearn.preprocessing.StandardScaler().fit(embs)
pkl.dump(open('preproc.pkl', 'wb'), preproc)
print(len(ngrams_list))

### Save mini-weights

In [ ]:
def save_mini_weights(num_top=1000,
    save_dir='~/mntv1/deep-fMRI/results/encoding/bert-10__ndel=4/UTS03'):
    """Call this once to save only the weights for the top voxels
    (All voxels is too big)
    Requires the full file weights.npz
    """
    # load weights
    weights_npz = np.load(join(save_dir, 'weights.npz'))
    weights = weights_npz['arr_0']
    weights = weights.reshape(4, -1, 768)
    # mean over delays dimension...
    weights = weights.mean(axis=0).squeeze()
    weights = weights.T  # make it (768, n_outputs)

    # load corrs
    corrs_val = np.load(join(save_dir, 'corrs.npz'))['arr_0']
    top_idxs = np.argsort(corrs_val)[::-1]

    # save top weights only
    weights = weights[:, top_idxs[:num_top]]
    pkl.dump(weights, open(join(save_dir, 'weights.pkl'), 'wb'))
save_mini_weights()